In [ ]:
%matplotlib inline

import numpy as np
import menpo.io as mio
from menpo.feature import greyscale, normalize_norm, normalize_std
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import PartsAAMBuilder, PartsAAMFitter

# AAMs

Load training data:

In [ ]:
training_images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/trainset/', verbose=True, 
                           max_images=100):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(200)
    labeller(i, 'PTS', ibug_face_66)
    training_images.append(i)

In [ ]:
visualize_images(training_images)

Load test data:

In [ ]:
test_images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/testset/', verbose=True, 
                           max_images=10):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(200)
    labeller(i, 'PTS', ibug_face_66)
    test_images.append(i)

In [ ]:
visualize_images(test_images)

Build Parts-AAM:

In [ ]:
builder = PartsAAMBuilder(features=greyscale, 
                          diagonal=100, 
                          norm_func=normalize_norm)

aam = builder.build(training_images, 
                    group='ibug_face_66', 
                    verbose=True)

Create Parts-AAM fitter:

In [ ]:
fitter = PartsAAMFitter(aam, 
                        n_shape=[3, 12], 
                        n_appearance=100)

Fit test data:

In [ ]:
np.random.seed(seed=0)

fitter_results = []
for j, i in enumerate(test_images):
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitter.perturb_shape(gt_s, noise_std=0.05)
    fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=20, map_inference=False)
    fr.downscale = 0.5
    fitter_results.append(fr)
    print 'Image: ', j
    print fr

In [ ]:
visualize_fitting_results(fitter_results)